### This notebook perfoms the following tasks:
##### 1) read Historical-Settlement-Reports.zip from DATA.GOV.AU
##### 2) ingest a selected report into pandas dataframe ("Local Government Area of Permanent Settlers (All Streams) with a Date of Settlement* between 01 January 2016 and 31 December 2016" in this example)
##### 3) GeoCode LGA with lat,long using Google Maps Geocoding API. Note you will need to use your own Google API key

In [1]:
import requests, zipfile, io
import pandas as pd
import re

In [2]:
api_key = 'put your Google Maps Geocoding API key here'

In [3]:
#get ZIP archive with Historical-Settlement-Reports from DATA.GOV.AU
read_zip = requests.get('https://data.gov.au/dataset/8d1b90a9-a4d7-4b10-ad6a-8273722c8628/resource/27adc111-5296-4dc4-884f-3cfbc33b02f3/download/historical-settlement-reports.zip')

In [4]:
archive = zipfile.ZipFile(io.BytesIO(read_zip.content))

In [5]:
def list_all_elements_of_a_list(list_name):
    for i in range (0,len(list_name)):
        print(i,":",list_name[i])

In [6]:
#archive.namelist() lists all files in archive.
#the code below lists all files together with list element index

list_all_elements_of_a_list(archive.namelist())

0 : statebymigrationstreamoct09tosep14.pdf
1 : agebymigrationstreamjan10todec14.docx
2 : agebymigrationstreamjan10todec14.pdf
3 : agebymigrationstreammarch2010toapril2015.docx
4 : agebymigrationstreammarch2010toapril2015.pdf
5 : agebymigrationstreamoct09tosep14.docx
6 : agebymigrationstreamoct09tosep14.pdf
7 : countryofbirthbymigrationstreamapril2010tomarch2015.docx
8 : countryofbirthbymigrationstreamapril2010tomarch2015.pdf
9 : countryofbirthbymigrationstreamjan10todec14.docx
10 : countryofbirthbymigrationstreamjan10todec14.pdf
11 : countryofbirthbymigrationstreamoct09tosep14.docx
12 : countryofbirthbymigrationstreamoct09tosep14.pdf
13 : ethnicitybymigrationstreamapril2010tomarch2015.docx
14 : ethnicitybymigrationstreamapril2010tomarch2015.pdf
15 : ethnicitybymigrationstreamjan10todec14.docx
16 : ethnicitybymigrationstreamjan10todec14.pdf
17 : ethnicitybymigrationstreamoct09tosep14.docx
18 : ethnicitybymigrationstreamoct09tosep14.pdf
19 : genderbymigrationstreamapril2010tomarch2015.do

In [7]:
#choose the file to read from the archive
i=91

In [8]:
file=archive.extract(archive.namelist()[i])
file

'/Users/alexanderfetisov/Documents/Savvi/ipynb/Settlement Data Reports January to December 2016.xlsx'

In [9]:
#read the file into dataframe
report = pd.ExcelFile(file)

In [10]:
#list all sheets of the excel file
list_all_elements_of_a_list(report.sheet_names)

0 : Caveats
1 : Settlement Date
2 : Age Group
3 : Country of Birth
4 : Current States
5 : Ethnicity
6 : Gender
7 : Local Government Area
8 : Main Language
9 : Religion


In [11]:
# read a specific sheet to DataFrame
sheet_id = 7

#skip the first 11 rows in the Excel sheet
skip_rows = 11 

#skip the first 11 rows in the Excel sheet
read_columns = [0,1,2,3,4,5]

In [12]:
#ingest sheet into dataframe
lga = report.parse(report.sheet_names[sheet_id], skiprows=skip_rows, usecols=read_columns, skip_footer=1)

In [13]:
#replace spaces in columns' header with _
lga.columns = lga.columns.str.replace('\s+', '_')

In [14]:
#Extract exact LGA name. Also convert 'Unincorporated <state>' into <state>.

def get_state(lga_name):
    state_name =  re.search('^.*(?=(\s\())', lga_name)
    unincorp_name =  re.search('Unincorporated', lga_name)
    
    # If state_name exists, extract and return it.
    if state_name:
        return state_name.group()
    # If lga_name conctains "Unicorporated", extract and return only State name.
    elif unincorp_name:
        return re.search('[^ ]* (.*)', lga_name).group(1)
    # in all other cases return original value
    else:
        return lga_name

In [15]:
lga['LGA'] = lga['Local_Government_Area'].apply(get_state)

In [16]:
lga

,Local_Government_Area,Humanitarian,Family,Skilled,Grand_Total,%_of_Total,LGA
0,Not Recorded,346,12991,17719,31056,0.118457,Not Recorded
1,Brisbane (C),963,4522,8748,14233,0.054289,Brisbane
2,Fairfield (C),4759,1569,323,6651,0.025369,Fairfield
3,Parramatta (C),389,1502,3577,5468,0.020857,Parramatta
4,Melbourne (C),60,896,4364,5320,0.020292,Melbourne
5,Sydney (C),27,1377,3899,5303,0.020227,Sydney
6,Hume (C),2415,782,1225,4422,0.016867,Hume
7,Wyndham (C),494,1358,2501,4353,0.016604,Wyndham
8,Greater Dandenong (C),441,1806,2056,4303,0.016413,Greater Dandenong
9,Unincorporated ACT,347,1241,2586,4174,0.015921,ACT


In [17]:
#define function to Geocode LGAs using Google Maps Geocode API
#length of API response is used as a criteria to identify bad requests, e.g. for non-existent addresses.

def get_coord(lga_name):

    url = r'https://maps.googleapis.com/maps/api/geocode/json?address='+lga_name+'+Australia&key='+api_key
    response = requests.get(url)
    if len(response.content) > 100:
        json_data = pd.read_json(response.content, orient='columns')
        api_resp=pd.io.json.json_normalize(json_data.results)
        api_resp.columns = api_resp.columns.str.replace('\.+', '_')
        temp = pd.DataFrame(api_resp)
        return [temp.geometry_location_lat[0], temp.geometry_location_lng[0], temp.formatted_address[0]]
    else: 
        return 'Bad_address'

In [18]:
#Iterate through the list of LGAs and for each LGA define Lat, Lng and formatted address

row_counter=0

for row in lga.itertuples():
    geocode_api_resp = get_coord(row.LGA)
    if geocode_api_resp != 'Bad_address':
        lga.at[row_counter, 'Lat'] = geocode_api_resp[0]
        lga.at[row_counter, 'Lng'] = geocode_api_resp[1]
        lga.at[row_counter, 'Formatted_address'] = geocode_api_resp[2]
    else:
        lga.at[row_counter, 'Lat'] = geocode_api_resp
        lga.at[row_counter, 'Lng'] = geocode_api_resp
        lga.at[row_counter, 'Formatted_address'] = geocode_api_resp
    row_counter += 1

In [19]:
lga

,Local_Government_Area,Humanitarian,Family,Skilled,Grand_Total,%_of_Total,LGA,Lat,Lng,Formatted_address
0,Not Recorded,346,12991,17719,31056,0.118457,Not Recorded,Bad_address,Bad_address,Bad_address
1,Brisbane (C),963,4522,8748,14233,0.054289,Brisbane,-27.4698,153.025,"Brisbane QLD, Australia"
2,Fairfield (C),4759,1569,323,6651,0.025369,Fairfield,-33.8703,150.956,"Fairfield NSW 2165, Australia"
3,Parramatta (C),389,1502,3577,5468,0.020857,Parramatta,-33.815,151.001,"Parramatta NSW 2150, Australia"
4,Melbourne (C),60,896,4364,5320,0.020292,Melbourne,-37.8136,144.963,"Melbourne VIC, Australia"
5,Sydney (C),27,1377,3899,5303,0.020227,Sydney,-33.8688,151.209,"Sydney NSW, Australia"
6,Hume (C),2415,782,1225,4422,0.016867,Hume,-35.387,149.17,"Hume ACT 2620, Australia"
7,Wyndham (C),494,1358,2501,4353,0.016604,Wyndham,-15.4825,128.123,"Wyndham WA 6740, Australia"
8,Greater Dandenong (C),441,1806,2056,4303,0.016413,Greater Dandenong,-38.0061,145.204,"Greater Dandenong, VIC, Australia"
9,Unincorporated ACT,347,1241,2586,4174,0.015921,ACT,-35.4735,149.012,"Australian Capital Territory, Australia"


In [20]:
lga.to_csv('/Users/alexanderfetisov/Documents/personal/lga_2016.csv') #write results into csv